# 05 - Time-to-Failure (TTF) Model Training

This notebook trains a neural network regression model to predict how many hours remain before a battery failure. This predictive maintenance capability helps anticipate battery issues before they occur.


## Disable GPU Usage

Configure TensorFlow to use CPU only for consistent execution.


In [1]:
import os
import warnings

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='keras')

## Import Required Libraries

Import all necessary libraries for data processing, model building, and evaluation.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import joblib
import subprocess
import shutil
import json

## Load Data and Calculate Time Before Failure

Load the battery data and calculate the time remaining until failure for each timestamp by measuring the difference from the maximum timestamp in the dataset.


In [3]:
df = pd.read_csv("../data/battery_data.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], format='ISO8601')
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600

## Select Features and Target Variable

Choose the most relevant battery parameters as input features and the calculated time-before-failure as the target for regression.


In [4]:
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

## Split and Normalize Data

Split the data into training and testing sets, then normalize the features to have zero mean and unit variance.


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Build Regression Neural Network

Create a neural network with three hidden layers for predicting continuous time-to-failure values.


In [6]:
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1) 
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

## Train the Model

Train the regression model for 50 epochs, using the test data for validation during training.


In [7]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0210 - mae: 0.1166 - val_loss: 0.0133 - val_mae: 0.0936
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0106 - mae: 0.0797 - val_loss: 0.0075 - val_mae: 0.0659
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0058 - mae: 0.0553 - val_loss: 0.0059 - val_mae: 0.0568
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0046 - mae: 0.0462 - val_loss: 0.0060 - val_mae: 0.0533
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0036 - mae: 0.0413 - val_loss: 0.0053 - val_mae: 0.0509
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0038 - mae: 0.0412 - val_loss: 0.0042 - val_mae: 0.0433
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0031 - mae: 0.0372 - val_loss: 0.0054 - val_mae: 0.0507
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0033 - mae: 0.0368 - val_loss: 0.0041 - val_mae: 0.0436
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.002

## Save Model and Scaler

Save the trained TTF model and scaler for later use in predictions.


In [8]:
os.makedirs("models", exist_ok=True)
scaler_path = "models/ttf_scaler.pkl"
joblib.dump(scaler, scaler_path)
model.save("models/battery_ttf_model.keras")

## Export Model to TensorFlow SavedModel Format

Export the model in TensorFlow's SavedModel format for conversion to OpenVINO IR.


In [ ]:
try:
    model.export("models/battery_ttf_model")
except AttributeError:
    model.save("models/battery_ttf_model", save_format='tf')

## Convert Model to OpenVINO IR Format

Convert the TensorFlow model to OpenVINO IR for optimized inference on Intel hardware.


In [ ]:
os.makedirs("models/battery_ttf_model_ir", exist_ok=True)

subprocess.run(
    [
        "ovc",
        "models/battery_ttf_model",
        "--output_model", "models/battery_ttf_model_ir/time-to-failure"
    ],
    capture_output=True,
    text=True,
    check=True
)

## Prepare Model for Serving

Organize the IR model files into a versioned directory structure for OpenVINO Model Server.


In [11]:

def prepare_ir_model_for_serving(ir_model_path, output_path, version=1):
    version_path = os.path.join(output_path, str(version))
    os.makedirs(version_path, exist_ok=True)
    
    for item in os.listdir(ir_model_path):
        if item.endswith(('.xml', '.bin')):
            src = os.path.join(ir_model_path, item)
            dst = os.path.join(version_path, item)
            shutil.copy2(src, dst)
    
    return version_path

serving_path = "models/serving/time-to-failure"
ir_model_path = "models/battery_ttf_model_ir"

model_version_path = prepare_ir_model_for_serving(ir_model_path, serving_path, version=1)
